In [14]:
import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import nltk
import string
from nltk.corpus import stopwords

In [15]:
import json
import pandas as pd
df = pd.read_excel(r'C:\Users\admin\Desktop\traini\newsdata.xlsx')

print(df)

                                              headline  \
0    Exclusive: Boeing deliveries to China delayed ...   
1    Not all Fed officials favored tapering of bala...   
2    S&P 500 to end 2024 around current levels afte...   
3    Cancer victims sue Johnson & Johnson over 'fra...   
4    Fed shifts talk to 'scenarios' as policy grows...   
..                                                 ...   
171  Bird Flu Cases Spur Retail Trader Rush Into Va...   
172  KKR-Backed Simon & Schuster Asks to Delay to F...   
173  Wind towers crumpled after Iowa wind farm suff...   
174  Charles Schwab to Roll Out Alternatives Platfo...   
175  KKR, TPG Said to Weigh PropertyGuru Options In...   

                           category  
0                   Regulatory News  
1                   Regulatory News  
2              Stock market update   
3    Bankruptcy/ financial distress  
4                   Regulatory News  
..                              ...  
171            Stock market updates  

In [16]:
top_10_categories = df['category'].value_counts().nlargest(10).index
print("Top 10 categories:", top_10_categories)
# Filtering data for top 10 categories
data = df[['category', 'headline']][df['category'].isin(
top_10_categories)].reset_index(drop=True)
display(data['category'].value_counts())

Top 10 categories: Index(['Regulatory News', 'Stock market updates', 'Merger & Acquisition',
       'financial results', 'Industry & Economic Updates',
       'Industry and Economic updates', 'Investment',
       'Corporate Legal Affairs', 'Currency News',
       'International Trade Updates'],
      dtype='object', name='category')


category
Regulatory News                  34
Stock market updates             32
Merger & Acquisition             18
financial results                13
Industry & Economic Updates      12
Industry and Economic updates    11
Investment                        9
Corporate Legal Affairs           8
Currency News                     7
International Trade Updates       6
Name: count, dtype: int64

In [17]:
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')

def preprocess_text(text):
    '''Preprocess text by making it lowercase, removing text in square brackets,removing links, removing punctuation, and removing words containing numbers.'''
    return re.sub('\[.*?\]|\w*\d\w*|https?://\S+|www\.\S+|<.*?>+|[%s]' % re.escape(string.punctuation), '', str(text).lower())

def eliminate_stopwords(text):
    '''Remove stopwords from the text'''
    stop_words = stopwords.words('english')
    words = text.split(' ')
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

def apply_stemming(sentence):
    stemmer = nltk.SnowballStemmer("english")
    return ' '.join(stemmer.stem(word) for word in sentence.split(' '))

def preprocess_and_clean(sentence):
    '''Preprocess and clean the text'''
    cleaned_text = preprocess_text(sentence)
    stop_words = stopwords.words('english')
    removed_stopwords_text = ' '.join(word for word in cleaned_text.split(' ') if word not in stop_words)
    stemmed_text = ' '.join(apply_stemming(word) for word in removed_stopwords_text.split(' '))
    return stemmed_text

data['clean_text'] = data['headline'].apply(preprocess_text)
data['clean_text_without_stopwords'] = data['clean_text'].apply(eliminate_stopwords)
data['stemmed_text'] = data['clean_text'].apply(apply_stemming)
data['preprocessed_text'] = data['clean_text'].apply(preprocess_and_clean)

<>:11: SyntaxWarning: invalid escape sequence '\['
<>:11: SyntaxWarning: invalid escape sequence '\['
C:\Users\admin\AppData\Local\Temp\ipykernel_3956\4203945148.py:11: SyntaxWarning: invalid escape sequence '\['
  return re.sub('\[.*?\]|\w*\d\w*|https?://\S+|www\.\S+|<.*?>+|[%s]' % re.escape(string.punctuation), '', str(text).lower())
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer()
Xtrain = data.sample(frac=0.9, random_state=42)
Ytest = data.drop(Xtrain.index)
# fit_transform for train data
tfid_X_train = vectorizer.fit_transform(Xtrain['clean_text'])
tfid_y_train = Xtrain['category']
# transform for test data
tfid_X_test = vectorizer.transform(Ytest['clean_text'])
tfid_y_test = Ytest['category']

In [30]:
from sklearn.linear_model import LogisticRegression
X_train, X_val, Y_train, Y_val = train_test_split(tfid_X_train, tfid_y_train, test_size=0.3, random_state=40)
model = LogisticRegression(random_state=40)
model.fit(X_train, Y_train)

LogisticRegression(random_state=40)

In [39]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=25, criterion='entropy', random_state=40)

# Fit the model to the training data
model.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=25, random_state=40)

In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
y_pred = model.predict(X_val)
acc = accuracy_score(Y_val, y_pred)
print(f'Accuracy: {acc}')
precision = precision_score(Y_val, y_pred, average='weighted')
print(f'Precision: {precision}')
# Calculate recall
recall = recall_score(Y_val, y_pred, average='weighted')
print(f'Recall: {recall}')

Accuracy: 0.5121951219512195
Precision: 0.5853658536585366
Recall: 0.5121951219512195


c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
